# Pokemon pipeline

Note: This notebook runs on Python 3.6 and uses UbiOps CLient Library 3.2.0.

If you run this entire notebook after filling in your access token, the pipeline and all the necessary models will be deployed to your UbiOps environment. You can thus check your environment after running to explore. You can also check the individual steps in this notebook to see what we did exactly and how you can adapt it to your own use case.

We recommend to run the cells step by step, as some cells can take a few minutes to finish.

## Establishing a connection with your UbiOps environment
Add your API token. Then we will provide a project name, deployment name and deployment version name. Afterwards we initialize the client library. This way we can deploy the two pipelines to your environment.

In [ ]:
API_TOKEN = '<INSERT API_TOKEN WITH PROJECT EDITOR RIGHTS>'
PROJECT_NAME= '<INSERT PROJECT NAME IN YOUR ACCOUNT>'
DEPLOYMENT_VERSION='v1'

import ubiops
import shutil

client = ubiops.ApiClient(ubiops.Configuration(api_key={'Authorization': API_TOKEN}, 
                                               host='https://api.ubiops.com/v2.1'))
api = ubiops.CoreApi(client)

## Creating the deployments



### Pokemon matcher
I found a [dataset on Kaggle](https://www.kaggle.com/abcsds/pokemon) that has statistics for every Pokémon, which we can use to match names to statistics. When dealing with customers instead of Pokémon this would be replaced by for example your CRM.

In this deployment we do the following steps: 
- Read the input Pokémon
- Read our file with all the Pokémon stats
- Select only the Pokémon we got as input
- Export this selection to a CSV file

See the actual code in the following cell.

In [ ]:
%load pokemon_matcher/deployment.py


Now we create a deployment and a deployment version for the package in the cell above. 

In [ ]:
# zip the models
shutil.make_archive('pokemon_matcher', 'zip', '.','pokemon_matcher')

deployment_template = ubiops.DeploymentCreate(
    name='pokemon-matcher',
    description='Match pokemon names to their stats',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(
            name='input_pokemon',
            data_type='array_string',
        )
    ],
    output_fields=[
        ubiops.DeploymentOutputFieldCreate(
            name='output_pokemon',
            data_type='blob'
        )
    ],
    labels={'demo': 'pokemon-pipeline'}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

#create the version
version_template = ubiops.DeploymentVersionCreate(
    version='v1',
    language='python3.6',
    memory_allocation=512,
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800 # = 30 minutes
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-matcher',
    data=version_template
)

#upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-matcher',
    version='v1',
    file='pokemon_matcher.zip'
)
print(file_upload_result.success)

### Pokemon sorter
We need to sort these Pokémon based on the best stats, we can start with the CSV from the Pokémon matcher step.

In this deployment we perform the following steps: 
- Read the input CSV(from the matcher step)
- Sort them based on their stats(higher is better)
- Export this as a CSV file

See the actual code in the following cell.

In [ ]:
%load pokemon_sorter/deployment.py


Now we create a deployment and a deployment version for the package in the cell above. 

In [ ]:
# zip the models
shutil.make_archive('pokemon_sorter', 'zip', '.','pokemon_sorter')

deployment_template = ubiops.DeploymentCreate(
    name='pokemon-sorter',
    description='Sort pokemon based on their stats',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(
            name='input_pokemon',
            data_type='blob',
        )
    ],
    output_fields=[
        ubiops.DeploymentOutputFieldCreate(
            name='output_pokemon',
            data_type='blob'
        )
    ],
    labels={'demo': 'pokemon-pipeline'}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

#create the version
version_template = ubiops.DeploymentVersionCreate(
    version='v1',
    language='python3.6',
    memory_allocation=512,
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800 # = 30 minutes
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-sorter',
    data=version_template
)

#upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-sorter',
    version='v1',
    file='pokemon_sorter.zip'
)
print(file_upload_result.success)

### Pokemon visualizer
I found a great [code snippet](https://www.kaggle.com/wenxuanchen/pokemon-visualization-radar-chart-t-sne) online for creating Pokémon stat visualizations, just like I remember seeing them on my old “Gameboy color”.

It is not really important to understand what’s happening in this visualization code, just that we can visualize the Pokémon we got from our previous step. The visualisation is then outputted as a PDF.

See the actual code in the following cell.

In [ ]:
%load pokemon_matcher/deployment.py


Now we create a deployment and a deployment version for the package in the cell above. 

In [ ]:
# zip the models
shutil.make_archive('pokemon_vis', 'zip', '.','pokemon_vis')

deployment_template = ubiops.DeploymentCreate(
    name='pokemon-vis',
    description='Visualize the results.',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(
            name='input_pokemon',
            data_type='blob',
        )
    ],
    output_fields=[
        ubiops.DeploymentOutputFieldCreate(
            name='output_pokemon',
            data_type='blob'
        )
    ],
    labels={'demo': 'pokemon-pipeline'}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

#create the version
version_template = ubiops.DeploymentVersionCreate(
    version='v1',
    language='python3.6',
    memory_allocation=512,
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800 # = 30 minutes
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-vis',
    data=version_template
)

#upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name='pokemon-vis',
    version='v1',
    file='pokemon_vis.zip'
)
print(file_upload_result.success)

## Creating the Pokemon pipeline

So right now we have three deployments. We want to tie these blocks together. We can use pipelines for that. Let's create a pipeline that takes the same input as the pokemon_sorter deployment.

In [ ]:
pipeline_name = "pokemon-pipeline"

pipeline_template = ubiops.PipelineCreate(
    name=pipeline_name,
    description='',
    input_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(
            name='input_pokemon',
            data_type='array_string',
        )
    ],
    labels={'demo': 'pokemon-pipeline'}
)

api.pipelines_create(
    project_name=PROJECT_NAME,
    data=pipeline_template
)

In [ ]:
PIPELINE_VERSION = DEPLOYMENT_VERSION

pipeline_template = ubiops.PipelineVersionCreate(
    version=PIPELINE_VERSION
)

api.pipeline_versions_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=pipeline_template)

We have a pipeline, now we just need to add our components to it and connect it.

**IMPORTANT**: only run the next cells once your deployments have finished building and are available. Otherwise you will get an error like: "error":"Version is not available: The version is currently in the building stage"

If you get this error, check in the UI if your model is ready and then rerun the cell below.

In [ ]:
from time import sleep

#Check if deployments are ready
status1 = 'building'
status2 = 'building'
status3 = 'building'

print("Checking if deployments are available")

while (status1 != 'available' and 'unavailable' not in status1) or (status2 != 'available' and 'unavailable' not in status2) :    
    version_status1 = api.deployment_versions_get(       
        project_name=PROJECT_NAME,
        deployment_name='pokemon-vis',
        version='v1',   
    )    
    status1 = version_status1.status
    
    version_status2 = api.deployment_versions_get(       
        project_name=PROJECT_NAME,
        deployment_name='pokemon-matcher',
        version='v1',    
    )   
    status2 = version_status2.status
    
    version_status3 = api.deployment_versions_get(       
        project_name=PROJECT_NAME,
        deployment_name='pokemon-sorter',
        version='v1',      
    )   
    status3 = version_status3.status
    sleep(1)
    
print("Done")


#Create pipeline objects
object_template = ubiops.PipelineVersionObjectCreate(
    name="pokemon-matcher",
    reference_name="pokemon-matcher",
    version='v1'
)

api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name="pokemon-pipeline", version=PIPELINE_VERSION, data=object_template)

object_template = ubiops.PipelineVersionObjectCreate(
    name="pokemon-sorter",
    reference_name="pokemon-sorter",
    version='v1'
)

api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name="pokemon-pipeline", version=PIPELINE_VERSION, data=object_template)

object_template = ubiops.PipelineVersionObjectCreate(
    name="pokemon-vis",
    reference_name="pokemon-vis",
    version='v1'
)

result = api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name="pokemon-pipeline", version=PIPELINE_VERSION, data=object_template)

print(result)

In [ ]:
# Connecting the components

# Start -> pokemon-matcher
attachment_template = ubiops.AttachmentsCreate(
    destination_name='pokemon-matcher',
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name='pipeline_start',
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='input_pokemon', destination_field_name='input_pokemon')]
        )
    ]
)

api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name='pokemon-pipeline', 
    version=PIPELINE_VERSION,
    data=attachment_template
)

# pokemon-matcher -> pokemon-sorter
attachment_template = ubiops.AttachmentsCreate(
    destination_name='pokemon-sorter',
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name='pokemon-matcher',
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output_pokemon', destination_field_name='input_pokemon')]
        )
    ]
)

api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name='pokemon-pipeline', 
    version=PIPELINE_VERSION,
    data=attachment_template
)

# pokemon-sorter -> pokemon-vis
attachment_template = ubiops.AttachmentsCreate(
    destination_name='pokemon-vis',
    sources=[
        ubiops.AttachmentSourcesCreate(
            source_name='pokemon-sorter',
            mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output_pokemon', destination_field_name='input_pokemon')]
        )
    ]
)

result = api.pipeline_version_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name='pokemon-pipeline', 
    version=PIPELINE_VERSION,
    data=attachment_template
)

print(result)

## Pokemon pipeline done!
If you check in your UbiOps account under pipeline you will find a pokemon-pipeline with our components in it and connected. Let's make a request to it. You can also make a request in the UI with the "create direct request button".

This might take a while since the models will need a cold start as they have never been used before.

## Making a request and exploring further
You can go ahead to the Web App and take a look in the user interface at what you have just built. If you want you can create a request to the pipeline. Just input a few Pokemon names. For example:

```
[
'Pikachu',
'Charmander'
]
```

For any questions, feel free to reach out to us via the customer service portal: https://ubiops.atlassian.net/servicedesk/customer/portals